# MIDI-Notes-Representations-MIDI (Ver 1.0)

***

### Based on the repo/code of Music X Lab at NYU Shanghai:

### https://github.com/music-x-lab/POP909-Dataset

***

### This is WIP, so please check back for updates and improvements

***

#### Project Los Angeles

#### Tegridy Code 2021

# Setup Environment

In [ ]:
#@title Install dependecies

!pip install pretty_midi
!pip install utils
!git clone https://github.com/asigalov61/tegridy-tools/

In [ ]:
#@title Import needed modules and create IO dirs
import pickle
import os
import sys
# import utils
import numpy as np
import pretty_midi as pyd
from pretty_midi import Note

os.chdir('/content/tegridy-tools/tegridy-tools')
import TMIDI
os.chdir('/content')

if not os.path.exists('/content/Dataset'):
  os.mkdir('/content/Dataset')

if not os.path.exists('/content/Output'):
  os.mkdir('/content/Output')

In [ ]:
#@title Download special Tegridy Piano MIDI dataset

#@markdown Works best stand-alone/as-is for the optimal results
%cd /content/Dataset/

!wget 'https://github.com/asigalov61/Tegridy-MIDI-Dataset/raw/master/Tegridy-Piano-CC-BY-NC-SA.zip'
!unzip -j '/content/Dataset/Tegridy-Piano-CC-BY-NC-SA.zip'
!rm '/content/Dataset/Tegridy-Piano-CC-BY-NC-SA.zip'

%cd /content/

# Setup classes and functions

In [ ]:
#@title Setup MIDI processing functions
'''
This is the data processing script
============
It will allow you to quickly process the MIDI Files into the Google Magenta's music representation 
    as like [Music Transformer](https://magenta.tensorflow.org/music-transformer) 
            [Performance RNN](https://magenta.tensorflow.org/performance-rnn).

'''

total = 0
def process_midi(path):
    global total
    data = pyd.PrettyMIDI(path)
    main_notes = []
    acc_notes = []
    for ins in data.instruments:
        acc_notes.extend(ins.notes)
    for i in range(len(main_notes)):
        main_notes[i].start = round(main_notes[i].start,2)
        main_notes[i].end = round(main_notes[i].end,2)
    for i in range(len(acc_notes)):
        acc_notes[i].start = round(acc_notes[i].start,2)
        acc_notes[i].end = round(acc_notes[i].end,2)
    main_notes.sort(key = lambda x:x.start)
    acc_notes.sort(key = lambda x:x.start)
    mpr = TMIDI.Tegridy_RPR_MidiEventProcessor()
    repr_seq = mpr.encode(acc_notes)
    total += len(repr_seq)
    return repr_seq
from pprint import pprint
def process_all_midis(midi_root, save_dir):
    save_py = []
    midi_paths = [d for d in os.listdir(midi_root)]
    i = 0
    out_fmt = '{}-{}.data'
    for path in midi_paths:
        pprint(path)
        filename = midi_root + path
        try:
            data = process_midi(filename)
        except KeyboardInterrupt:
            print(' Abort')
            return
        except EOFError:
            print('EOF Error')
            return
        save_py.append(data)
    # pprint(save_py, compact=True)    
    save_py = np.array(save_py, dtype='object')
    print('Total number of MIDIs:', save_py.size)
    np.save("/content/Output/notes_representations.npy", save_py)

# Use examples

In [ ]:
#@title Convert a sample MIDI to the RPR and print the result
repr_seq = process_midi('/content/tegridy-tools/tegridy-tools/seed.mid')
print(repr_seq)

In [ ]:
#@title Convert created RPR back to MIDI file
# Create a PrettyMIDI object
output = pyd.PrettyMIDI()
# Create an Instrument instance for a piano instrument
output_program = pyd.instrument_name_to_program('Acoustic Grand Piano')
piano = pyd.Instrument(program=output_program)

# Decode representations into Pretty_MIDI notes
mpr = TMIDI.Tegridy_RPR_MidiEventProcessor()
notes = mpr.decode(repr_seq)

# Add notes to the Pretty MIDI object
piano.notes.extend(notes)

output.instruments.append(piano)
# Write out the MIDI data
output.write('/content/Output/output.mid')
print('Saved as /content/Output/output.mid')

In [ ]:
#@title Convert ALL MIDIs in the Dataset folder to the RPR Numpy Array
# Process all MIDIs in the Dataset folder into Notes representations Numpy Array
process_all_midis("/content/Dataset/","/content/Output/")

In [ ]:
#@title Print resulting Numpy Array for debug/eval
np.load('/content/Output/notes_representations.npy', allow_pickle=True)

In [ ]:
#@title Print first element (MIDI RPR) in the resulting Numpy Array for debug/eval
pprint(np.load('/content/Output/notes_representations.npy', allow_pickle=True)
[0], compact=True)

# Congrats! You've done it!

# Enjoy! :)